In [1]:
import sys, platform

# ----------------------------------------------------------------------------------- 
# NOTE:  Update the path below to point to the directory where `ub_camera.py` is saved.
# ----------------------------------------------------------------------------------- 
CODE_PATH = "/home/cmurray3/Projects/IE-482-582/spring2026/Projects/code"
sys.path.append(CODE_PATH)

import ub_camera
import ub_utils
import cv2

In [2]:
# Start a camera instance
# Initialize `CameraUSB` Class
port = ub_utils.findOpenPort(8001, options=range(8000,8040))

paramDict = {'res_rows':480, 'res_cols':640, 'fps_target':30, 'outputPort': port}
apiPref   = None   # cv2.CAP_V4L2 # None
device    = 0 # 'https://10.83.17.66:8002/stream.mjpg'  # 0      # '/dev/video0' # 0          # '/dev/video0'

try:
    system = platform.system()
    if (system == 'Windows'):
        sslPath   = f'{CODE_PATH}\ssl'
    else:
        # For Linux/Mac
        sslPath   = f'{CODE_PATH}/ssl'
except Exception as e:
    print(f'Error - Could not set sslPath: {e}')

camera = ub_camera.CameraUSB(paramDict = paramDict, 
                             device = device, 
                             apiPref = apiPref,
                             sslPath = sslPath,
                             showFPS=False)

camera.start(startStream=True, port=paramDict['outputPort'])

# Hide the frames-per-second text from the video feed
camera.showFPS = False

print(f'Visit https://localhost:{paramDict["outputPort"]}/stream.mjpg')

Port 8001 is already in use.
Logger:  No topic name/type found.  Just using print()
LOGGER: {'res_rows': 480, 'res_cols': 640, 'fps_target': 30, 'outputPort': 8000}
Visit https://localhost:8000/stream.mjpg
Camera Opened? True


**Run the next line when you're ready to shut down the camera:**

In [6]:
camera.stop()

In [7]:
camera.keepStreaming

False

--- 

## Calibration
- See the `calibration_example.ipynb` notebook.
- NOTE:  You might want to calibrate the camera for `320x240`, too.

In [7]:
# This is copied from the camera calibration code.  See `calibration_example.ipynb`:
camera.intrinsics = { "640x480": {"fx": 613.9267755271052, "fy": 617.2876757419133, "cx": 326.06379688638367, "cy": 226.4726965669937, "dist": [-0.040671732389409375, 0.2205460570452358, -0.008313365917653356, 0.0025141234454979433, -0.32871689004906784]  } }
camera.intrinsics = camera._getIntrinsics()
camera.intrinsics

{'640x480': {'dist': array([-0.04067173,  0.22054606, -0.00831337,  0.00251412, -0.32871689]),
  'matrix': array([[613.92677553,   0.        , 326.06379689],
         [  0.        , 617.28767574, 226.47269657],
         [  0.        ,   0.        ,   1.        ]])}}

--- 

## ArUco Tags

In [3]:
# Specify the size of the ArUco tag in inches (or enter `None` if unknown)
TAG_SIZE_INCHES = None    #  4 + 3/16   

In [4]:
def aruco_post_poses(argsDict):
    # This function gets called each time an aruco detection is run
    idName  = argsDict['idName']

    if (TAG_SIZE_INCHES is not None):
        # Adjust based on resolution
        res = f'{camera.res_cols}x{camera.res_rows}'
        cameraMatrix = camera.intrinsics[res]['matrix']
        dist = camera.intrinsics[res]['dist']

        # ********************************************
        # Specify the size of the marker, in [meters]
        # ********************************************
        ml = ub_utils.inches2meters(TAG_SIZE_INCHES)  

        objPoints = np.array([[-ml/2,  ml/2, 0], 
                              [ ml/2,  ml/2, 0], 
                              [ ml/2, -ml/2, 0], 
                              [-ml/2, -ml/2, 0]])
        
    corners = camera.aruco[idName].deque[0]['corners']
    for i in range(len(corners)):
        # centers give the center point, in pixels, of the tag.
        print(f"id: {camera.aruco[idName].deque[0]['ids'][i]}")
        print(f"\tcenter: {camera.aruco[idName].deque[0]['centers'][i]}")

        if (TAG_SIZE_INCHES is not None):
            (ret, rvecs, tvecs) = ub_utils.arucoFindPose(objPoints, corners[i], cameraMatrix, dist, flags=cv2.SOLVEPNP_IPPE_SQUARE)
            # print(f"{ret=}, {rvecs=}, {tvecs=}.")
            if (ret):
                # rvecs is the 3D rotation vector.  
                # I don't think it's human interpretable, so we won't print it here. 

                # tvecs in the x/y/z translation of the marker from the origin (camera).
                # It's in [meters], since we specified `ml` in [meters].
                # tvecs[0] (x) is the distance left (-) or right (+) from the camera.
                # tvecs[1] (y) is the distance above (-) or below (+) the camera.
                # tvecs[2] (z) is the distance away from the camera.
                print(f"\tdistance [inches]: x: {ub_utils.meters2inches(tvecs[0])}, y: {ub_utils.meters2inches(tvecs[1])}, z: {ub_utils.meters2inches(tvecs[2])}")

In [5]:
# Specify what type of ArUco tag you have:
ARUCO_DICTIONARY = 'DICT_APRILTAG_36h11'

In [6]:
camera.addAruco(idName=ARUCO_DICTIONARY, 
                fps_target=5, 
                postFunction=aruco_post_poses, 
                postFunctionArgs={'idName': ARUCO_DICTIONARY}, 
                configOverrides={}, 
                ids_of_interest=None)  # default is None, or provide a list of IDs to track

LOGGER: Starting ArUco DICT_APRILTAG_36h11 thread at 5 fps
id: 10
	center: [309 435]
id: 11
	center: [462 330]
id: 10
	center: [361 323]
id: 9
	center: [466 205]
id: 11
	center: [432 354]
id: 9
	center: [430 224]
id: 8
	center: [329 218]
id: 10
	center: [324 350]
id: 11
	center: [418 347]
id: 10
	center: [312 342]
id: 9
	center: [418 217]
id: 8
	center: [318 211]
id: 11
	center: [407 347]
id: 10
	center: [304 342]
id: 9
	center: [409 218]
id: 8
	center: [310 213]
id: 11
	center: [394 347]
id: 9
	center: [397 219]
id: 10
	center: [290 342]
id: 8
	center: [298 214]
id: 11
	center: [373 371]
id: 10
	center: [270 366]
id: 9
	center: [377 241]
id: 8
	center: [278 236]
id: 11
	center: [382 373]
id: 10
	center: [280 366]
id: 9
	center: [386 246]
id: 8
	center: [290 240]
id: 11
	center: [397 371]
id: 9
	center: [399 247]
id: 10
	center: [295 364]
id: 8
	center: [305 243]
id: 10
	center: [314 370]
id: 9
	center: [415 254]
id: 8
	center: [322 250]
id: 11
	center: [414 376]
id: 11
	center: [437 3

**Run the next cell when you're ready to stop the ArUco detection:**

In [8]:
camera.aruco[ARUCO_DICTIONARY].stop()

LOGGER: Stopping ArUco DICT_APRILTAG_36h11 thread.


--- 

## Barcodes

In [19]:
def postBarcode(argsDict):
    # print(camera.barcode['default'].deque[0])
    for i in range(len(camera.barcode['default'].deque[0]['data'])): 
        print(f"""data: {camera.barcode['default'].deque[0]['data'][i]},  
                codeType: {camera.barcode['default'].deque[0]['codeTypes'][i]},
                quality: {camera.barcode['default'].deque[0]['qualities'][i]},
                corners: {camera.barcode['default'].deque[0]['corners'][i]}""")
  

In [20]:
camera.addBarcode(fps_target=5,
                  postFunction=postBarcode)

LOGGER: Starting barcode default thread at 5 fps
data: 108292762,  
                codeType: CODE128,
                quality: 1,
                corners: [(64, 259), (64, 259)]


**Run the next cell when you're ready to stop the barcode detection:**

In [21]:
camera.barcode['default'].stop()

LOGGER: Stopping barcode default thread.
LOGGER: Stopping barcode default thread.
LOGGER: Stopping barcode default thread.
LOGGER: Stopping barcode default thread.


--- 
## Face Detection

In [20]:
# Need to indicate where the OpenCV face detection models are saved.
try:
    system = platform.system()
    if (system == 'Windows'):
        modelPath   = f'{CODE_PATH}\cv2_dnn_models'
    else:
        # For Linux/Mac
        modelPath   = f'{CODE_PATH}/cv2_dnn_models'
except Exception as e:
    print(f'Error - Could not set face detection modelPath: {e}')

print(f'The OpenCV face detection models should be saved in the `{modelPath}` directory.')

The OpenCV face detection models should be saved in the `/home/cmurray3/Projects/IE-482-582/spring2025/Projects/code/cv2_dnn_models` directory.


In [21]:
def postFaceDetect(argsDict):
    # print(camera.facedetect['default'].deque[0])
    for i in range(len(camera.facedetect['default'].deque[0]['confidence'])): 
        print(f"{i} - confidence: {camera.facedetect['default'].deque[0]['confidence'][i]}, corners: {camera.facedetect['default'].deque[0]['corners'][i]}")


In [22]:
camera.addFaceDetect(fps_target=5,
                     postFunction=postFaceDetect, 
                     conf_threshold=0.7, 
                     dnn='caffe',    # 'caffe' (fp16) or 'pb' (8bit)
                     device='cpu', 
                     modelPath=modelPath)

LOGGER: Starting facedetect default thread at 5 fps
0 - confidence: 0.8832073211669922, corners: [(256, 256), (367, 438)]
0 - confidence: 0.9041808247566223, corners: [(255, 256), (366, 437)]
0 - confidence: 0.9090292453765869, corners: [(255, 256), (366, 437)]
0 - confidence: 0.9597670435905457, corners: [(260, 254), (371, 431)]
0 - confidence: 0.9735339283943176, corners: [(268, 254), (380, 430)]
0 - confidence: 0.9801608920097351, corners: [(269, 255), (380, 432)]
0 - confidence: 0.9773929715156555, corners: [(269, 255), (380, 431)]
0 - confidence: 0.9723621606826782, corners: [(266, 253), (377, 431)]
0 - confidence: 0.9479912519454956, corners: [(258, 252), (369, 431)]
0 - confidence: 0.8882012963294983, corners: [(251, 254), (364, 431)]
0 - confidence: 0.8886810541152954, corners: [(248, 252), (358, 427)]
0 - confidence: 0.9062338471412659, corners: [(252, 251), (364, 426)]
0 - confidence: 0.9434531331062317, corners: [(256, 251), (368, 422)]
0 - confidence: 0.9868587255477905, co

0 - confidence: 0.9471254944801331, corners: [(306, 242), (429, 457)]
0 - confidence: 0.9484015703201294, corners: [(305, 242), (430, 455)]
0 - confidence: 0.9371916055679321, corners: [(306, 243), (430, 453)]
0 - confidence: 0.9463715553283691, corners: [(305, 243), (430, 453)]
0 - confidence: 0.9498353004455566, corners: [(306, 243), (431, 453)]
0 - confidence: 0.9564929604530334, corners: [(306, 243), (431, 454)]
0 - confidence: 0.957192063331604, corners: [(307, 243), (431, 455)]
0 - confidence: 0.9601184725761414, corners: [(307, 243), (431, 455)]
0 - confidence: 0.9519088268280029, corners: [(307, 243), (431, 454)]
0 - confidence: 0.9585315585136414, corners: [(308, 243), (432, 454)]
0 - confidence: 0.9679420590400696, corners: [(309, 243), (433, 457)]
0 - confidence: 0.9636630415916443, corners: [(309, 243), (433, 456)]
0 - confidence: 0.9638124108314514, corners: [(315, 243), (436, 454)]
0 - confidence: 0.9764654636383057, corners: [(312, 243), (436, 455)]
0 - confidence: 0.971

**Run the next cell when you're ready to stop the face detection:**

In [24]:
camera.facedetect['default'].stop()

LOGGER: Stopping FaceDetect default thread.


--- 
## Ultralytics
The following options are documented:
- Detect
- Pose
- Oriented Bounding Box (obb)
- Segment (mask)
- Track (can be applied to `Detect`, `Pose`, and `Segment`)

### Detect

In [3]:
def postUltralyticsDetect(argsDict):
    idName = argsDict['idName']
    results = argsDict['results']
    
    for result in results:
        '''
        xywh = result.boxes.xywh  # center-x, center-y, width, height
        xywhn = result.boxes.xywhn  # normalized
        xyxy = result.boxes.xyxy  # top-left-x, top-left-y, bottom-right-x, bottom-right-y
        xyxyn = result.boxes.xyxyn  # normalized
        names = [result.names[cls.item()] for cls in result.boxes.cls.int()]  # class name of each box
        confs = result.boxes.conf  # confidence score of each box    
        '''

        for i in range(0, len(result.boxes.cls)):
            # print(int(result.boxes.cls[i].item())
            # print(camera.ultralytics[idName].model.names[int(result.boxes.cls[i].item())])
            # print(result.boxes.conf[i].item(), result.boxes.xyxy[i].tolist())
            print(f'{result.names[int(result.boxes.cls[i].item())]} ({result.boxes.conf[i].item()}), {result.boxes.xyxy[i].tolist()}')

In [4]:
camera.addUltralytics(idName="detect", 
                      model_name="yolo11n.pt", 
                      conf_threshold=0.75, 
                      postFunction=postUltralyticsDetect)

LOGGER: Starting Ultralytics detect thread at 30 fps


/home/cmurray3/.local/lib/python3.8/site-packages/torch/cuda/__init__.py:128: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0


person (0.9062300324440002), [185.67266845703125, 211.1949462890625, 604.9562377929688, 479.51177978515625]
person (0.9211145639419556), [182.03582763671875, 208.1078338623047, 607.3900146484375, 479.43682861328125]
person (0.9202371835708618), [182.30413818359375, 208.00506591796875, 607.3364868164062, 479.43292236328125]
person (0.9197365641593933), [182.2332763671875, 207.92922973632812, 607.8096923828125, 479.4153137207031]
person (0.9216938614845276), [182.20196533203125, 207.5211181640625, 607.7946166992188, 479.40863037109375]
person (0.9221044182777405), [182.2054443359375, 207.99972534179688, 608.339599609375, 479.4241638183594]
person (0.9219133853912354), [182.2139892578125, 208.033203125, 609.810302734375, 479.4295654296875]
person (0.9196762442588806), [182.037109375, 207.9822998046875, 608.7572021484375, 479.458740234375]
person (0.9215047359466553), [182.29258728027344, 207.87875366210938, 609.0828857421875, 479.4454650878906]
person (0.9195607304573059), [182.4095458984

person (0.9166655540466309), [183.01463317871094, 207.26287841796875, 604.8380126953125, 479.46502685546875]
person (0.9158965945243835), [182.91766357421875, 207.40237426757812, 604.7515869140625, 479.4701232910156]
person (0.9171674847602844), [182.7740478515625, 207.53330993652344, 603.0717163085938, 479.4656982421875]
person (0.9178571701049805), [182.707275390625, 207.27517700195312, 603.6416625976562, 479.4465026855469]
person (0.9171895980834961), [182.6571807861328, 207.51510620117188, 603.5709838867188, 479.4408264160156]
person (0.9170852303504944), [182.607177734375, 207.30230712890625, 604.3265380859375, 479.45916748046875]
person (0.916483461856842), [182.7320556640625, 207.32623291015625, 604.29150390625, 479.458251953125]
person (0.9193795919418335), [182.6951904296875, 207.35467529296875, 603.9849243164062, 479.4515380859375]
person (0.9177840948104858), [182.81829833984375, 207.33938598632812, 604.9158325195312, 479.4662780761719]
person (0.9171915054321289), [182.8028

person (0.9152301549911499), [182.97906494140625, 207.8338623046875, 603.9549560546875, 479.4775390625]
person (0.9160488843917847), [182.8634033203125, 207.89987182617188, 606.5208129882812, 479.4770812988281]
person (0.9152449369430542), [183.01947021484375, 208.05752563476562, 604.626953125, 479.4833679199219]
person (0.9167047142982483), [183.12127685546875, 208.07675170898438, 605.7125244140625, 479.4796447753906]
person (0.9171264171600342), [183.03436279296875, 208.35382080078125, 603.24365234375, 479.47979736328125]
person (0.9172974228858948), [183.1400146484375, 208.47866821289062, 604.598876953125, 479.4799499511719]
person (0.9155490398406982), [183.0373077392578, 208.28656005859375, 602.3539428710938, 479.49456787109375]
person (0.9163402915000916), [183.16612243652344, 208.25933837890625, 604.852294921875, 479.49761962890625]
person (0.9163015484809875), [183.31005859375, 208.05953979492188, 603.26806640625, 479.4833068847656]
person (0.9176221489906311), [183.45416259765

person (0.9134505987167358), [182.6329803466797, 208.8527374267578, 607.2106323242188, 479.49053955078125]
person (0.9150166511535645), [182.63922119140625, 208.47146606445312, 608.063232421875, 479.4747619628906]
person (0.9139386415481567), [182.64385986328125, 208.62557983398438, 606.9071044921875, 479.4855651855469]
person (0.9152734279632568), [182.51434326171875, 208.78863525390625, 607.0509033203125, 479.4786376953125]
person (0.9149805903434753), [182.77703857421875, 208.66749572753906, 605.7177734375, 479.4837646484375]
person (0.9146386384963989), [182.989990234375, 208.91091918945312, 606.9945678710938, 479.4933776855469]
person (0.9151033759117126), [182.7406005859375, 208.6802978515625, 606.4139404296875, 479.49334716796875]
person (0.9141876697540283), [182.92413330078125, 208.68011474609375, 606.63720703125, 479.4912109375]
person (0.9131626486778259), [182.9512939453125, 208.69384765625, 604.6163940429688, 479.49212646484375]
person (0.91451096534729), [182.764892578125

In [ ]:
camera.ultralytics['detect'].model.names

In [ ]:
camera.ultralytics['detect'].drawBox   = True
camera.ultralytics['detect'].drawLabel = True

In [5]:
camera.ultralytics['detect'].stop()

person (0.9083804488182068), [192.71051025390625, 209.96759033203125, 606.2239379882812, 479.47845458984375]
LOGGER: Stopping Ultralytics detect thread.
person (0.9085239768028259), [192.67413330078125, 209.7715301513672, 606.2459106445312, 479.47637939453125]
LOGGER: Stopping Ultralytics detect thread.
LOGGER: Stopping ultralytics detect thread.
LOGGER: Stopping Ultralytics detect thread.


### Pose

In [13]:
def postUltralyticsPose(argsDict):
    idName = argsDict['idName']
    results = argsDict['results']
    
    '''
    `keypoints` should have 17 elements:
    0: Nose, 1: Left Eye, 2: Right Eye, 3: Left Ear, 4: Right Ear,
    5: Left Shoulder, 6: Right Shoulder, 7: Left Elbow, 8: Right Elbow, 9: Left Wrist, 10: Right Wrist,
    11: Left Hip, 12: Right Hip, 13: Left Knee, 14: Right Knee, 15: Left Ankle, 16: Right Ankle
    '''
    
    for result in results:
        if (result.keypoints.has_visible):
            print(f'conf: {result.keypoints.conf.tolist()}, keypoints: {result.keypoints.xy.tolist()} \n')


In [ ]:
camera.addUltralytics(idName="pose", 
                      model_name="yolo11n-pose.pt", 
                      conf_threshold=0.75,  
                      postFunction=postUltralyticsPose, 
                      drawBox = False, drawLabel=True)

In [20]:
camera.ultralytics['pose'].drawBox   = False
camera.ultralytics['pose'].drawLabel = False

In [19]:
camera.ultralytics['pose'].stop()

LOGGER: Stopping Ultralytics pose thread.


### OBB

In [21]:
def postUltralyticsObb(argsDict):
    idName = argsDict['idName']
    results = argsDict['results']
    
    for result in results:
        if (result.obb):
            for i in range(0, len(result.obb.cls)):
                    print(f'{result.names[int(result.obb.cls[i].item())]} ({result.obb.conf[i].item()}), Center: {result.obb.xywhr[i][0:2].tolist()}')        

In [22]:
camera.addUltralytics(idName="obb", 
                      model_name="yolo11n-obb.pt", 
                      conf_threshold=0.65,  
                      postFunction=postUltralyticsObb, 
                      drawBox = True, drawLabel=True)

LOGGER: Starting Ultralytics obb thread at 30 fps
storage tank (0.6884562373161316), Center: [526.8790283203125, 322.8346252441406]
plane (0.7441368699073792), Center: [30.149080276489258, 271.05657958984375]
plane (0.7527572512626648), Center: [313.65765380859375, 176.14439392089844]
plane (0.6662600040435791), Center: [64.23978424072266, 212.36898803710938]
plane (0.7216538190841675), Center: [73.19515991210938, 218.69818115234375]
plane (0.667588472366333), Center: [330.06658935546875, 187.00880432128906]
plane (0.7769538760185242), Center: [385.6541748046875, 207.299560546875]
plane (0.689613938331604), Center: [386.3738708496094, 209.32839965820312]
plane (0.7816662192344666), Center: [389.3519287109375, 211.91400146484375]
plane (0.7355049848556519), Center: [389.34210205078125, 213.29702758789062]
plane (0.7128258347511292), Center: [388.9755554199219, 214.71270751953125]
plane (0.8470562100410461), Center: [391.6231384277344, 216.44540405273438]
plane (0.8009462356567383), Cent

plane (0.8413069844245911), Center: [394.7250671386719, 269.6236877441406]
ship (0.7033405900001526), Center: [184.95718383789062, 404.49114990234375]
plane (0.851328432559967), Center: [395.6539611816406, 271.0973815917969]
ship (0.6920074820518494), Center: [184.50743103027344, 407.785400390625]
ship (0.6626908183097839), Center: [171.54595947265625, 394.229736328125]
ship (0.6515731811523438), Center: [204.16549682617188, 421.5318603515625]
plane (0.8803566098213196), Center: [402.80657958984375, 277.7911071777344]
plane (0.7025067806243896), Center: [83.37124633789062, 410.7205505371094]
plane (0.8108454346656799), Center: [419.278564453125, 288.9842224121094]
ship (0.7246977090835571), Center: [427.0494689941406, 242.531005859375]
plane (0.7979350090026855), Center: [447.0497131347656, 291.781982421875]
ship (0.7249875068664551), Center: [196.90614318847656, 448.1374206542969]
plane (0.8636087775230408), Center: [457.1932067871094, 292.4229431152344]
ship (0.7464613914489746), Cen

plane (0.8197342753410339), Center: [495.485107421875, 298.0320739746094]
ship (0.7736046314239502), Center: [235.55772399902344, 455.9024658203125]
ship (0.7255872488021851), Center: [259.1622619628906, 472.6728515625]
ship (0.7198886275291443), Center: [211.72027587890625, 426.30218505859375]
ship (0.7141871452331543), Center: [222.50059509277344, 439.81732177734375]
ship (0.651073694229126), Center: [308.4193115234375, 367.51556396484375]
plane (0.8517655730247498), Center: [492.888916015625, 297.3030700683594]
ship (0.7295646071434021), Center: [220.8076629638672, 439.51055908203125]
ship (0.7261496186256409), Center: [234.1455078125, 455.09124755859375]
ship (0.7211971879005432), Center: [257.5775146484375, 471.86474609375]
ship (0.7080767154693604), Center: [209.50732421875, 426.0565490722656]
ship (0.6632955074310303), Center: [382.6361083984375, 397.9594421386719]
plane (0.8360518217086792), Center: [492.772216796875, 296.7442932128906]
ship (0.721626877784729), Center: [210.12

In [ ]:
camera.ultralytics['obb'].model.names

In [24]:
camera.ultralytics['obb'].stop()

LOGGER: Stopping Ultralytics obb thread.


### Segment

In [3]:
def postUltralyticsSegment(argsDict):
    idName = argsDict['idName']
    results = argsDict['results']
    
    for result in results:
        for i in range(0, len(result.boxes.cls)):
            try:
                print(f'{result.names[int(result.boxes.cls[i].item())]} ({result.boxes.conf[i].item()}), {result.boxes.xyxy[i].tolist()}')   
            except Exception as e:
                print(f'Error: {e}')

In [ ]:
camera.addUltralytics(idName="segment", 
                      model_name="yolo11n-seg.pt", 
                      conf_threshold=0.65,  
                      postFunction=postUltralyticsSegment, 
                      drawBox = False, drawLabel=True, 
                      maskOutline = False)

In [ ]:
camera.ultralytics['segment'].maskOutline = True

In [28]:
camera.ultralytics['segment'].model.names

{0: 'person',
 1: 'bicycle',
 2: 'car',
 3: 'motorcycle',
 4: 'airplane',
 5: 'bus',
 6: 'train',
 7: 'truck',
 8: 'boat',
 9: 'traffic light',
 10: 'fire hydrant',
 11: 'stop sign',
 12: 'parking meter',
 13: 'bench',
 14: 'bird',
 15: 'cat',
 16: 'dog',
 17: 'horse',
 18: 'sheep',
 19: 'cow',
 20: 'elephant',
 21: 'bear',
 22: 'zebra',
 23: 'giraffe',
 24: 'backpack',
 25: 'umbrella',
 26: 'handbag',
 27: 'tie',
 28: 'suitcase',
 29: 'frisbee',
 30: 'skis',
 31: 'snowboard',
 32: 'sports ball',
 33: 'kite',
 34: 'baseball bat',
 35: 'baseball glove',
 36: 'skateboard',
 37: 'surfboard',
 38: 'tennis racket',
 39: 'bottle',
 40: 'wine glass',
 41: 'cup',
 42: 'fork',
 43: 'knife',
 44: 'spoon',
 45: 'bowl',
 46: 'banana',
 47: 'apple',
 48: 'sandwich',
 49: 'orange',
 50: 'broccoli',
 51: 'carrot',
 52: 'hot dog',
 53: 'pizza',
 54: 'donut',
 55: 'cake',
 56: 'chair',
 57: 'couch',
 58: 'potted plant',
 59: 'bed',
 60: 'dining table',
 61: 'toilet',
 62: 'tv',
 63: 'laptop',
 64: 'mou

In [8]:
camera.ultralytics['segment'].stop()

LOGGER: Stopping Ultralytics segment thread.


### Track

In [9]:
def postUltralyticsTrack(argsDict):
    idName = argsDict['idName']
    results = argsDict['results']
    
    # print(idName)   # "track"
    for result in results:
        '''
        xywh = result.boxes.xywh  # center-x, center-y, width, height
        xywhn = result.boxes.xywhn  # normalized
        xyxy = result.boxes.xyxy  # top-left-x, top-left-y, bottom-right-x, bottom-right-y
        xyxyn = result.boxes.xyxyn  # normalized
        names = [result.names[cls.item()] for cls in result.boxes.cls.int()]  # class name of each box
        confs = result.boxes.conf  # confidence score of each box    
        '''
        for i in range(0, len(result.boxes.cls)):
            try:
                print(f'ID: {result.boxes.id[i].item()} - {result.names[int(result.boxes.cls[i].item())]} ({result.boxes.conf[i].item()}), {result.boxes.xyxy[i].tolist()}')                
            except Exception as e:
                print(f'Error: {e}')

In [10]:
# Tracking can be done with detect, pose, or segment models:
model_name = "yolo11n.pt"       # detect
# model_name = "yolo11n-pose.pt"
# model_name = "yolo11n-seg.pt"

camera.addUltralytics(idName="track", 
                      model_name=model_name, 
                      conf_threshold=0.65,  
                      postFunction=postUltralyticsTrack, 
                      drawBox = False, drawLabel=True)

LOGGER: Starting Ultralytics track thread at 30 fps
ID: 1.0 - person (0.6899388432502747), [215.70458984375, 204.28890991210938, 565.9527587890625, 479.5946960449219]
ID: 1.0 - person (0.7208012938499451), [224.048828125, 204.3238525390625, 565.4356079101562, 479.6250915527344]
ID: 1.0 - person (0.7290352582931519), [232.0320281982422, 204.40280151367188, 563.0572509765625, 479.58160400390625]
ID: 1.0 - person (0.7292996048927307), [234.98944091796875, 204.25294494628906, 566.7814331054688, 479.59991455078125]
ID: 1.0 - person (0.692925751209259), [233.7186737060547, 205.08314514160156, 567.6341552734375, 479.6534118652344]
ID: 1.0 - person (0.7654642462730408), [236.1581268310547, 204.61672973632812, 566.545654296875, 479.6298828125]
ID: 1.0 - person (0.7846347093582153), [235.38429260253906, 204.25048828125, 565.5188598632812, 479.59326171875]
ID: 1.0 - person (0.7642068266868591), [230.8628692626953, 203.91259765625, 566.7721557617188, 479.66058349609375]
ID: 1.0 - person (0.7969377

ID: 1.0 - person (0.8027042150497437), [250.8868408203125, 207.3205108642578, 560.8078002929688, 479.6289978027344]
ID: 1.0 - person (0.7099381685256958), [254.7388153076172, 206.7089385986328, 518.539306640625, 480.0]
ID: 1.0 - person (0.758437991142273), [254.96905517578125, 206.9671630859375, 559.9817504882812, 480.0]
ID: 1.0 - person (0.7408413290977478), [261.0968322753906, 206.02127075195312, 578.2030029296875, 479.8292541503906]
ID: 1.0 - person (0.779449462890625), [264.36932373046875, 205.33648681640625, 583.596435546875, 479.30523681640625]
ID: 1.0 - person (0.7746638655662537), [264.1317443847656, 204.1080322265625, 583.5905151367188, 479.2278137207031]
ID: 1.0 - person (0.7483105659484863), [268.49737548828125, 202.97227478027344, 585.4718017578125, 479.2923278808594]
ID: 1.0 - person (0.6782705783843994), [255.641845703125, 201.67446899414062, 586.0341796875, 479.09100341796875]
ID: 1.0 - person (0.7979127764701843), [255.67539978027344, 201.3741455078125, 586.082824707031

ID: 1.0 - person (0.8607358932495117), [266.1158752441406, 221.01739501953125, 583.51953125, 479.2218322753906]
ID: 1.0 - person (0.8676068782806396), [266.37847900390625, 220.93910217285156, 583.3568115234375, 479.2424011230469]
ID: 1.0 - person (0.8750854730606079), [266.8119201660156, 220.94793701171875, 582.5332641601562, 479.2740173339844]
ID: 1.0 - person (0.8635459542274475), [266.95391845703125, 220.72010803222656, 582.0441284179688, 479.2994689941406]
ID: 1.0 - person (0.8654294610023499), [266.6548156738281, 220.43014526367188, 581.683349609375, 479.3319396972656]
ID: 1.0 - person (0.8607076406478882), [266.03125, 220.27488708496094, 581.2681274414062, 479.3470764160156]
ID: 1.0 - person (0.8674383163452148), [265.75421142578125, 220.2604217529297, 582.0906982421875, 479.3828430175781]
ID: 1.0 - person (0.8712992668151855), [266.0941467285156, 220.61122131347656, 582.8711547851562, 479.391845703125]
ID: 1.0 - person (0.864587128162384), [266.3446044921875, 220.2003936767578, 

In [5]:
camera.ultralytics['track'].drawBox = False
camera.ultralytics['track'].drawLabel = True

In [12]:
camera.ultralytics['track'].stop()

LOGGER: Stopping Ultralytics track thread.


In [13]:
camera.stopStream()
camera.stop()

--- 
## ROI
- Probably won't document this function anytime soon

## Timelapse
- TBD

## Video from Pics
- TBD